In [5]:
import os

if "src" not in os.listdir():
    os.chdir("../../../")

from datetime import datetime, timezone, timedelta
import pandas_market_calendars as mcal
from tqdm import tqdm

from src.lib.alpaca_historical import AlpacaData
from src.lib.util import makedir_to

In [14]:
KEYS_PATH = "alpaca_config.json" # Path to Alpaca Keys in user system
symbols = ["SPY", "SPXS"] # What tickers do we want
end = datetime.now(timezone.utc)
start = end - timedelta(minutes=10)

offset = timedelta(seconds=4)
width = timedelta(seconds=0.1)

nyse = mcal.get_calendar("NYSE")

schedule = nyse.schedule(start_date=start, end_date=end)
valid_minutes = mcal.date_range(schedule, frequency="1T")   

In [15]:
data_dir = "data"

alpaca = AlpacaData(KEYS_PATH)

for symbol in symbols:
    df = None
    pbar = tqdm(valid_minutes)
    for minute in pbar:
        data = alpaca.get_quotes(symbol, minute + offset - width/2, minute + offset + width/2, False)
        if df is None:
            df = data
        else:
            df = df.append(data)
        
        time_str = df.index[-1].strftime("%y-%m-%d %H:%M:%S")
        pbar.set_description(f"{time_str} - {len(df)}")
        
    save_path = os.path.join(data_dir, symbol, "%s_%s.csv" % (symbol, "small_quotes"))
    makedir_to(save_path)
    df.to_csv(save_path)

21-09-24 17:38:04 - 536: 100%|██████████| 390/390 [00:41<00:00,  9.40it/s]
